In [1]:
import numpy as np
import pandas as pd
import random
import sys
import os
from tqdm import tqdm
sys.path.append("../../tools/")
import commontools
import ucTools
import funclib
import time

sys.path.append("../../")
import train

from sklearn.model_selection import train_test_split
from xgboost.sklearn import XGBClassifier
from sklearn.metrics import classification_report
from sklearn.metrics import f1_score, precision_score, recall_score
import joblib  # 将模型导出所需包

%load_ext autoreload
%autoreload 2


In [2]:
train = pd.read_feather('../../data/benchmark/data/train.feather')

In [45]:
train=train[train.isemzyme]

In [50]:
data_x = train.iloc[0:20000,4:1914]
data_y = np.array(train.functionCounts.iloc[0:20000])

In [53]:
data_y

array([1, 1, 1, ..., 1, 1, 1])

In [54]:
data_y = data_y-1

In [55]:
x_train, x_test, y_train, y_test = train_test_split(data_x,data_y,test_size=0.2,random_state=1)

In [56]:
def importance_features_top(model_str, model, x_train):
    """打印模型的重要指标，排名top50指标"""
    print("打印XGBoost重要指标")
    feature_importances_ = model.feature_importances_
    feature_names = x_train.columns
    importance_col = pd.DataFrame([*zip(feature_names, feature_importances_)],  columns=['features', 'weight'])
    importance_col_desc = importance_col.sort_values(by='weight', ascending=False)
    print(importance_col_desc.iloc[:50, :])

def xgboost_model(data_x,data_y):
    
    """用XGBoost进行建模，返回训练好的模型"""
    x_train, x_vali, y_train, y_vali = train_test_split(data_x,data_y,test_size=0.3,random_state=1)
    eval_set = [(x_train, y_train), (x_vali, y_vali)]
    
    xgboost_clf = XGBClassifier(min_child_weight=6, max_depth=15, objective='multi:softmax', num_class=11, use_label_encoder=False)
    print("-" * 100)
    print("几功能酶xgboost模型：", xgboost_clf)
    xgboost_clf.fit(x_train, y_train, eval_metric="mlogloss", eval_set=eval_set, verbose=True)
    # # 打印重要性指数
    importance_features_top('xgboost', xgboost_clf, x_train)
    # 保存模型
    joblib.dump(xgboost_clf, './XGBoost_model_v1.0')
    return xgboost_clf

def print_precison_recall_f1(y_true, y_pre):
    """打印精准率、召回率和F1值"""
    print("打印精准率、召回率和F1值")
    print(classification_report(y_true, y_pre))
    f1 = round(f1_score(y_true, y_pre, average='macro'), 2)
    p = round(precision_score(y_true, y_pre, average='macro'), 2)
    r = round(recall_score(y_true, y_pre, average='macro'), 2)
    print("Precision: {}, Recall: {}, F1: {} ".format(p, r, f1))

In [57]:
xgboost_clf = xgboost_model(x_train, y_train)

----------------------------------------------------------------------------------------------------
几功能酶xgboost模型： XGBClassifier(base_score=None, booster=None, colsample_bylevel=None,
              colsample_bynode=None, colsample_bytree=None, gamma=None,
              gpu_id=None, importance_type='gain', interaction_constraints=None,
              learning_rate=None, max_delta_step=None, max_depth=15,
              min_child_weight=6, missing=nan, monotone_constraints=None,
              n_estimators=100, n_jobs=None, num_class=11,
              num_parallel_tree=None, objective='multi:softmax',
              random_state=None, reg_alpha=None, reg_lambda=None,
              scale_pos_weight=None, subsample=None, tree_method=None,
              use_label_encoder=False, validate_parameters=None,
              verbosity=None)
[0]	validation_0-mlogloss:1.16518	validation_1-mlogloss:1.17082
[1]	validation_0-mlogloss:0.85412	validation_1-mlogloss:0.87449
[2]	validation_0-mlogloss:0.65743	v

In [18]:
eval_set = [(x_train, y_train), (x_test, y_test)]

In [69]:
pre_y_test = xgbost_clf.predict(x_test)

NameError: name 'xost_clf' is not defined

In [16]:
data_x

,f1,f2,f3,f4,f5,f6,f7,f8,f9,f10,...,f1891,f1892,f1893,f1894,f1895,f1896,f1897,f1898,f1899,f1900
0,0.000033,-0.004133,0.031950,-0.000895,0.015118,0.084028,-0.106755,-0.198294,-0.000912,0.010045,...,-0.001402,-0.064062,0.084055,-0.039602,-0.011034,-0.902350,0.153470,0.003805,0.032580,-0.034380
1,0.000181,0.234055,0.202590,-0.009154,0.257440,-0.008591,-0.039887,-0.009504,-0.005681,-0.014996,...,0.198902,-0.006073,-0.186542,0.023011,-0.113903,-0.074157,0.820054,0.045937,-0.036160,-0.028256
2,0.000188,0.292311,0.244053,-0.009429,0.271757,-0.005266,-0.078286,-0.007095,-0.006165,-0.020575,...,0.175442,0.065406,-0.041244,0.025119,-0.096559,-0.079137,0.789780,0.064275,-0.033467,-0.100108
3,0.000223,0.245537,0.244524,-0.006509,0.386281,0.054393,0.058923,-0.005218,-0.005565,-0.034299,...,0.157748,0.305814,-0.130900,0.045086,-0.247851,-0.131615,0.759615,0.015326,-0.045659,-0.123342
4,0.003292,-0.010891,0.011057,-0.004417,-0.259515,0.133029,-0.464210,-0.002968,-0.005446,0.249855,...,0.055954,-0.187867,0.014005,0.038685,0.390966,-0.139071,-0.140427,-0.000358,-0.071718,-0.002881
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
95,0.000804,0.070719,0.018773,-0.004329,0.881442,-0.000350,0.029116,-0.002443,-0.001216,-0.625112,...,0.003353,0.010496,-0.253492,0.009834,-0.471886,-0.143367,-0.066733,0.002626,-0.118797,-0.020916
96,0.002619,0.071596,0.039163,-0.007932,0.684397,0.001810,0.004194,-0.005130,-0.003821,-0.699007,...,0.002460,0.040557,-0.309668,0.009582,-0.497647,-0.113421,-0.026370,0.001468,-0.042494,-0.091926
97,0.003020,0.029630,0.064161,-0.001923,0.894576,-0.005141,-0.034928,-0.004715,-0.003473,-0.648881,...,-0.009807,0.017115,-0.186568,0.011872,-0.702187,-0.083997,-0.046591,0.002655,-0.052006,-0.054832
98,0.001743,-0.010931,0.036082,-0.004149,0.355944,0.046004,-0.081538,-0.007427,-0.001497,-0.138363,...,0.006606,0.081028,-0.314343,0.006154,-0.466908,-0.089873,-0.013782,0.005637,0.118699,-0.021342


In [72]:
pre_y_test = xgboost_clf.predict(x_test)
# 打印测试集的结果信息，包含precision、recall、f1-socre
print("-" * 30, "测试集", "-" * 30)
print_precison_recall_f1((y_test-1), pre_y_test)

------------------------------ 测试集 ------------------------------
打印精准率、召回率和F1值
              precision    recall  f1-score   support

          -1       0.00      0.00      0.00      3698
           0       0.05      0.85      0.09       214
           1       0.00      0.00      0.00        36
           2       0.00      0.00      0.00        16
           3       0.00      0.00      0.00        13
           4       0.00      0.00      0.00        10
           5       0.00      0.00      0.00         7
           6       0.00      0.00      0.00         3
           7       0.00      0.00      0.00         2
           8       0.00      0.00      0.00         1

    accuracy                           0.05      4000
   macro avg       0.00      0.08      0.01      4000
weighted avg       0.00      0.05      0.00      4000

Precision: 0.0, Recall: 0.08, F1: 0.01 


/home/shizhenkun/anaconda3/envs/py38/lib/python3.8/site-packages/sklearn/metrics/_classification.py:1245: UndefinedMetricWarning: Precision and F-score are ill-defined and being set to 0.0 in labels with no predicted samples. Use `zero_division` parameter to control this behavior.
  _warn_prf(average, modifier, msg_start, len(result))
/home/shizhenkun/anaconda3/envs/py38/lib/python3.8/site-packages/sklearn/metrics/_classification.py:1245: UndefinedMetricWarning: Precision and F-score are ill-defined and being set to 0.0 in labels with no predicted samples. Use `zero_division` parameter to control this behavior.
  _warn_prf(average, modifier, msg_start, len(result))
/home/shizhenkun/anaconda3/envs/py38/lib/python3.8/site-packages/sklearn/metrics/_classification.py:1245: UndefinedMetricWarning: Precision and F-score are ill-defined and being set to 0.0 in labels with no predicted samples. Use `zero_division` parameter to control this behavior.
  _warn_prf(average, modifier, msg_start, le

In [79]:
pre_y_test+1

array([1, 1, 1, ..., 1, 1, 1])

In [76]:
aa=xgboost_clf.predict_proba(x_test)

In [78]:
pd.DataFrame(aa)

,0,1,2,3,4,5,6,7,8,9
0,0.999536,0.000185,0.000097,0.000023,0.000031,0.000015,0.000012,0.000029,0.000036,0.000036
1,0.998039,0.001149,0.000033,0.000419,0.000101,0.000040,0.000040,0.000045,0.000065,0.000069
2,0.999494,0.000347,0.000066,0.000009,0.000007,0.000008,0.000006,0.000021,0.000020,0.000021
3,0.995280,0.003173,0.000951,0.000150,0.000058,0.000034,0.000051,0.000068,0.000110,0.000125
4,0.996984,0.001482,0.000407,0.000223,0.000129,0.000052,0.000100,0.000268,0.000172,0.000183
...,...,...,...,...,...,...,...,...,...,...
3995,0.995232,0.004096,0.000122,0.000077,0.000071,0.000041,0.000099,0.000065,0.000102,0.000096
3996,0.993236,0.005212,0.000091,0.000439,0.000164,0.000097,0.000136,0.000163,0.000231,0.000232
3997,0.997971,0.001500,0.000260,0.000053,0.000024,0.000014,0.000019,0.000053,0.000051,0.000054
3998,0.974866,0.018734,0.000197,0.002519,0.000347,0.000336,0.001450,0.000477,0.000536,0.000537
